In [33]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import re
import warnings

 
#Set Random seed
np.random.seed(522)

# Add the Data using pandas
Corpus = pd.read_csv(r'Data/Nepal_Eq.csv', encoding='latin-1')
print ("loadind data ... : Nepal_Earthquake")




loadind data ... : Nepal_Earthquake


#### Data Pre-processing - This will help in getting better results through the classification algorithms

In [34]:
#   Data Pre-processing - This will help in getting better results through the classification algorithms

#   Remove blank rows if any.
Corpus['text'].dropna(inplace=True)

#   Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]

#   Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]

#  Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.

# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV


for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word, tag_map[tag[0]])
            word_Final = re.sub("rt", "", word_Final)
            word_Final = re.sub("http", "", word_Final)
            word_Final = re.sub(":", "", word_Final)
            word_Final = re.sub("@", "", word_Final)
            word_Final = re.sub("#", "", word_Final)
            word_Final = re.sub("//", "", word_Final)
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index, 'text_final'] = str(Final_words)
    Corpus['text_final'].dropna(inplace=True)
#print(Corpus['text_final'].head())

####   Split the model into Train and Test Data set

In [35]:

Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'], Corpus['label'], test_size=0.3, random_state=3291)

#   Label encode the target variable  - This is done to transform Categorical data of string type in the data set into numerical values
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)


#### Vectorize the words by using TF-IDF Vectorizer 

In [36]:
#This is done to find how important a word in document is in comaprison to the corpus
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)



####  Now we can run different algorithms to classify out data check for accuracy

####  SVM Classifier 

In [37]:

# fit the training dataset on the classifier
SVM = svm.SVC(C=100, kernel='rbf', gamma=0.1)
SVM.fit(Train_X_Tfidf, Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM :")
print("SVM Accuracy Score -> ",  accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM f1_score -> ", f1_score(predictions_SVM, Test_Y)*100)

SVM :
SVM Accuracy Score ->  87.29198184568835
SVM f1_score ->  91.13924050632912


#### Naive Bayes   Classifier   

In [38]:

# fit the training dataset on the classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf, Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use  f1_score and accuracy_score, function 
print("Naive Bayes :")
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Test_Y)*100)
print("Naive Bayes f1_score -> ", f1_score(predictions_NB, Test_Y)*100)

Naive Bayes :
Naive Bayes Accuracy Score ->  85.62783661119516
Naive Bayes f1_score ->  90.49049049049049


####  Random Forest Classifier

In [39]:
 
rf = RandomForestClassifier()
rf.fit(Train_X_Tfidf, Train_Y)
RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
# predict the labels on validation dataset
predictions_RF = rf.predict(Test_X_Tfidf)
print("Random Forest ====================>")
# Use accuracy_score function to get the accuracy
print("RF Accuracy Score -> ", accuracy_score(predictions_RF, Test_Y)*100)
print("RF f1_score -> ", f1_score(predictions_RF, Test_Y)*100)

Random Forest ====================>
RF Accuracy Score ->  86.08169440242057
RF f1_score ->  90.49586776859503
